In [1]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import time

%matplotlib inline

# boilerplate code

In [17]:
#multiply multiplies two permutations written in disjoint format to get another in same format
#signed off
def elementList(a,b):
    elements = []
    for i in b:
        elements += [x for x in i]
    for j in a:
        elements += [y for y in j if y not in elements]
    return elements

def nextElement(element,theSet):
    for tupl in theSet:
        if element in tupl:
            for i in range(0,len(tupl)):
                if tupl[i]==element:
                    Next = tupl[(i+1)%len(tupl)]
                    return Next
    return element

def multiply(a,b):
    answer = []
    elements = elementList(a,b)
    unusedElements = elements[1:]
    tup = [elements[0]]
    length = 1
    while length<len(elements):
        first = tup[-1]
        second = nextElement(first, b)
        third = nextElement(second, a)
        if third in tup:
            if len(tup)>1:
                answer.append(tup)
            tup = [unusedElements[0]]
            unusedElements = unusedElements[1:]
            length+=1
        else:
            unusedElements.remove(third)
            tup.append(third)
            length+=1
    if len(tup)>1:
        answer.append(tup)
    return answer

In [20]:
#finds the inverse of a permutation
# signed off
def inverse(a):
    answer=[]
    for tupl in a:
        answer.append(tupl[::-1])
    return answer

In [7]:
#I think this works?
def uniqueTup(tupl):
    m=min(tupl)
    i=tupl.index(m)
    answer = tupl[i:] + tupl[0:i]
    return answer,m

def uniquePerm(a):
    answerList={}
    mList=[]
    answer=[]
    for x in a:
        tup,m = uniqueTup(x)
        answerList[m]=tup
        mList.append(m)
    mList.sort()
    for i in mList:
        answer.append(answerList[i])
    return answer

# Question 1

    Idea: We're trying to find the ccls of G
    Step 1: Find the ccls of the generators using the wildfire algorithm
    Step 2: If inverse of g not in a ccl, can easily generate another ccl
    

In [8]:
def conjugate(x,g):
    x_ = inverse(x)
    xg=multiply(x,g)
    return multiply(xg,x_)

In [32]:
#this seems like adequate proof that conjugate works - there is no reason for it not to work
a=[[2,3,4,5]]

for i in range(0,5):
    print(a)
    a=uniquePerm(conjugate([[1,2,3,4,5]],a))
print(a)

[[2, 3, 4, 5]]
[[1, 3, 4, 5]]
[[1, 2, 4, 5]]
[[1, 2, 3, 5]]
[[1, 2, 3, 4]]
[[2, 3, 4, 5]]


In [59]:
#just finds the conjugacy classes of the generators 
#kinda useless, also appears not to work
def conjClass(generators):
    cclList=[]
    for i in generators:
        cclI = wildfire(generators,i)
        cclList.append(cclI)
    return cclList

In [64]:
conjClass([[[1,2,3,5,4]],[[2,3,4,5]]])

[[[[1, 2, 3, 5, 4]], [[1, 3, 4, 2, 5]], [[1, 4, 5, 3, 2]], [[1, 5, 2, 4, 3]]],
 [[[1, 4, 3, 5]],
  [[2, 3, 4, 5]],
  [[1, 5, 4, 2]],
  [[1, 3, 2, 4]],
  [[1, 2, 5, 3]]]]

    Idea: next have to generate all ccls from a subset of representatives
    Step 1: find the ccls of the generator elements
    Step 2: use the wildfire algorithm again 

In [62]:
def wildfire(generators,node):
    ccl=[]
    nodeConj(node,generators,ccl)
    return ccl


#finds the conjugacy class of a node    
def nodeConj(node,generators,cclList):
    new=[]
    for i in generators:
        a = uniquePerm(conjugate(i,node))
#         print(node,i,a)
        if a not in cclList:
            cclList.append(a)
            new.append(a)
    for j in new:
        nodeConj(j,generators,cclList)        

In [65]:
wildfire([[[1,2,3,5,4]],[[2,3,4,5]]],[[2,4],[3,5]])

[[[1, 3], [4, 5]],
 [[2, 4], [3, 5]],
 [[1, 4], [2, 5]],
 [[1, 2], [3, 4]],
 [[1, 5], [2, 3]]]

In [81]:
def groupGen(generators,group,node):
    if node in group:
        return
    group.append(node)
    for g in generators:
        newNode=multiply(node,g)
        groupGen(generators,group,newNode)

In [83]:
group=[]
groupGen([[[1, 2, 3,4,5,6,7]],[[1,2]]],group,[])
print(len(group))

RecursionError: maximum recursion depth exceeded in comparison

    Basic idea: want to do this using a queue rather than recursion to ease breadth first search\
    While loop on things in the queue

In [85]:
def newGroupGen(generators):
    group = []
    queue = [[]]
    while len(queue)>0:
        node=queue[0]
        if node not in group:
            for g in generators:
                newNode = uniquePerm(multiply(g,node))
                queue.append(newNode)
            group.append(node)
        queue = queue[1:]
    return group

In [91]:
def newConjClass(generators,node):
    cclList= []
    queue = [node]
    while len(queue)>0:
        node = queue[0]
        if node not in cclList:
            cclList.append(node)
            for g in generators:
                newNode = uniquePerm(conjugate(g,node))
                queue.append(newNode)
        queue = queue[1:]
    return cclList

In [109]:
#technically works - could probably do to be faster
#potentially getting rid of totalCcl and just manually checking each time would be better
def newConjFinder(generators):
    cclList = []
    totalCcl = []
    group = []
    queue = [[]]
    while len(queue)>0:
        node = queue[0]
        if node not in group:
            group.append(node)
            if node not in totalCcl:
                newCcl = newConjClass(generators,node)
                totalCcl += newCcl
                cclList.append(newCcl)
            for g in generators:
                newNode = uniquePerm(multiply(g,node))
                queue.append(newNode)
        queue = queue[1:]
    return cclList

In [108]:
cclList = newConjFinder([[[1, 2, 3,4,5,6,7]],[[1,2]]])
for ccl in cclList:
    print(ccl[0],len(ccl))

[] 1
[[1, 2, 3, 4, 5, 6, 7]] 720
[[1, 2]] 21
[[2, 3, 4, 5, 6, 7]] 840
[[1, 3, 5, 7], [2, 4, 6]] 420
[[1, 4, 7, 3, 6], [2, 5]] 504
[[2, 4, 6], [3, 5, 7]] 280
[[1, 4, 7], [2, 5], [3, 6]] 210
[[2, 5], [3, 6], [4, 7]] 105
[[1, 7, 6, 5, 4]] 504
[[1, 2, 3]] 70
[[1, 2], [3, 4]] 105
[[1, 7, 6, 5]] 210
[[1, 2, 3], [4, 5]] 420
[[1, 2, 3, 4], [5, 6]] 630


In [92]:
newConjClass([[[1, 2, 3,5,4]],[[2,3,4,5]]],[[2,3,4,5]])
# newGroupGen([[[1, 2, 3,5,4]],[[2,3,4,5]]])

[[[2, 3, 4, 5]],
 [[1, 4, 3, 5]],
 [[1, 5, 4, 2]],
 [[1, 3, 2, 4]],
 [[1, 2, 5, 3]]]

In [72]:
#current bugs - missing out one of the conjugacy classes
def nodeCcl(generators, node, cclList,ticker):
#     print("new",node)
    for i in cclList:
        if node in i:
            return
    cclList.append(wildfire(generators,node))
    print(ticker,node)
    for j in generators:
#         print("old",node)
        new = uniquePerm(multiply(node, j))
        nodeCcl(generators, new, cclList,ticker+1)
        
def wildfire2(generators):
    cclList = conjClass(generators)
    for node in generators:
        nodeCcl(generators,node, cclList)
    return cclList

In [73]:
# need to get rid of extra counts
ccl=[]
nodeCcl([[[1, 2, 3,4,5,6,7]],[[1,2]]],[],ccl,0)
for i in ccl:
    print(i[0],len(i))
    
print("geronimo")
ccl=[]
nodeCcl([[[1, 2, 3,4,5,6,7]],[[1,2]]],[[1,2]],ccl,0)
for i in ccl:
    print(i[0],len(i))

0 []
1 [[1, 2, 3, 4, 5, 6, 7]]
2 [[1, 3, 4, 5, 6, 7]]
3 [[1, 2, 4, 6], [3, 5, 7]]
4 [[1, 4, 7, 2, 5], [3, 6]]
5 [[1, 5], [2, 4, 7], [3, 6]]
4 [[1, 4, 6], [3, 5, 7]]
1 [[1, 2]]
[] 1
[[1, 2, 3, 4, 5, 6, 7]] 720
[[1, 2, 4, 5, 6, 7]] 840
[[1, 4, 6], [2, 3, 5, 7]] 420
[[1, 3, 6, 2, 5], [4, 7]] 504
[[1, 3, 5], [2, 6], [4, 7]] 210
[[1, 4, 6], [2, 5, 7]] 280
[[2, 3]] 21
geronimo
0 [[1, 2]]
1 [[2, 3, 4, 5, 6, 7]]
2 [[1, 3, 5, 7], [2, 4, 6]]
3 [[1, 4, 7, 3, 6], [2, 5]]
4 [[1, 5, 2, 4, 7, 3, 6]]
1 []
[[2, 3]] 21
[[1, 3, 4, 5, 6, 7]] 840
[[1, 2, 4, 6], [3, 5, 7]] 420
[[1, 4, 7, 2, 5], [3, 6]] 504
[[1, 4, 7, 2, 6, 3, 5]] 720
[] 1


In [61]:
ccl=[]
nodeCcl([[[1, 2, 3,5,4]],[[2,3,5,4]]],[],ccl)
print(ccl)

[] [[1, 2, 3, 5, 4]] []
[] [[2, 3, 5, 4]] []
[] [[1, 2, 3, 5, 4]] []
[] [[2, 3, 5, 4]] []
[[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]]
[[1, 2, 3, 5, 4]] [[2, 3, 5, 4]] [[1, 3, 5, 4, 2]]
[[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]]
[[1, 2, 3, 5, 4]] [[2, 3, 5, 4]] [[1, 3, 5, 4, 2]]
[[1, 3, 5, 4, 2]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 5, 4]]
[[1, 3, 5, 4, 2]] [[2, 3, 5, 4]] [[1, 5, 4, 2, 3]]
[[1, 3, 2, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 3, 4]]
[[1, 3, 2, 5, 4]] [[2, 3, 5, 4]] [[1, 5, 3, 4, 2]]
[[1, 2, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 4, 5]]
[[1, 2, 5, 3, 4]] [[2, 3, 5, 4]] [[1, 3, 4, 5, 2]]
[[1, 2, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 2, 3, 5]]
[[1, 2, 3, 4, 5]] [[2, 3, 5, 4]] [[1, 3, 5, 2, 4]]
[[1, 4, 2, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 4, 2]]
[[1, 4, 2, 3, 5]] [[2, 3, 5, 4]] [[1, 2, 3, 5, 4]]
[[1, 3, 5, 2, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 4, 3]]
[[1, 3, 5, 2, 4]] [[2, 3, 5, 4]] [[1, 5, 4, 3, 2]]
[[1, 2, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 3, 4]]
[[1, 2, 5, 4, 3]

[[1, 2, 4, 5, 3]] [[1, 2, 3, 5, 4]] [[1, 4, 5, 2, 3]]
[[1, 2, 4, 5, 3]] [[2, 3, 5, 4]] [[1, 3, 2, 4, 5]]
[[1, 4, 5, 3, 2]] [[1, 2, 3, 5, 4]] [[1, 4, 5, 3, 2]]
[[1, 4, 5, 3, 2]] [[2, 3, 5, 4]] [[1, 2, 4, 5, 3]]
[[1, 4, 5, 2, 3]] [[1, 2, 3, 5, 4]] [[1, 4, 3, 5, 2]]
[[1, 4, 5, 2, 3]] [[2, 3, 5, 4]] [[1, 2, 4, 3, 5]]
[[1, 4, 3, 5, 2]] [[1, 2, 3, 5, 4]] [[1, 5, 4, 3, 2]]
[[1, 4, 3, 5, 2]] [[2, 3, 5, 4]] [[1, 2, 5, 4, 3]]
[[1, 3, 4, 2, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 2, 5]]
[[1, 3, 4, 2, 5]] [[2, 3, 5, 4]] [[1, 5, 2, 3, 4]]
[[1, 5, 4, 3, 2]] [[1, 2, 3, 5, 4]] [[1, 5, 3, 2, 4]]
[[1, 5, 4, 3, 2]] [[2, 3, 5, 4]] [[1, 4, 2, 5, 3]]
[[2, 5, 1, 3]] [[1, 2, 3, 5, 4]] [[2, 5, 3, 4]]
[[2, 5, 1, 3]] [[2, 3, 5, 4]] [[1, 5, 3, 4]]
[[2, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 5]]
[[2, 5, 3, 4]] [[2, 3, 5, 4]] [[2, 3, 4, 5]]
[[1, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 2, 5]]
[[1, 3, 4, 5]] [[2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 4, 2, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 2]]
[[1, 4, 2, 5]] [[2, 3, 5, 4]] [[1, 2, 3, 4]

[[2, 3], [4, 5]] [[2, 3, 5, 4]] [[2, 4], [3, 5]]
[[1, 4], [3, 5]] [[1, 2, 3, 5, 4]] [[1, 2], [4, 5]]
[[1, 4], [3, 5]] [[2, 3, 5, 4]] [[1, 2], [4, 5]]
[[1, 2], [4, 5]] [[1, 2, 3, 5, 4]] [[1, 4], [2, 3]]
[[1, 2], [4, 5]] [[2, 3, 5, 4]] [[1, 3], [2, 4]]
[[1, 4], [2, 3]] [[1, 2, 3, 5, 4]] [[1, 2], [3, 5]]
[[1, 4], [2, 3]] [[2, 3, 5, 4]] [[1, 2], [3, 5]]
[[1, 2], [3, 5]] [[1, 2, 3, 5, 4]] [[2, 3], [4, 5]]
[[1, 2], [3, 5]] [[2, 3, 5, 4]] [[1, 3], [4, 5]]
[[1, 3], [2, 4]] [[1, 2, 3, 5, 4]] [[1, 3], [2, 5]]
[[1, 3], [2, 4]] [[2, 3, 5, 4]] [[1, 5], [2, 3]]
[[1, 3], [2, 5]] [[1, 2, 3, 5, 4]] [[2, 5], [3, 4]]
[[1, 3], [2, 5]] [[2, 3, 5, 4]] [[1, 5], [3, 4]]
[[2, 5], [3, 4]] [[1, 2, 3, 5, 4]] [[1, 5], [3, 4]]
[[2, 5], [3, 4]] [[2, 3, 5, 4]] [[2, 5], [3, 4]]
[[1, 5], [3, 4]] [[1, 2, 3, 5, 4]] [[1, 5], [2, 4]]
[[1, 5], [3, 4]] [[2, 3, 5, 4]] [[1, 4], [2, 5]]
[[1, 5], [2, 4]] [[1, 2, 3, 5, 4]] [[1, 3], [2, 4]]
[[1, 5], [2, 4]] [[2, 3, 5, 4]] [[1, 4], [2, 3]]
[[2, 1, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 2

[[1, 2, 4, 3, 5]] [[2, 3, 5, 4]] [[1, 3, 2, 5, 4]]
[[1, 5, 4, 2, 3]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 2, 4]]
[[1, 5, 4, 2, 3]] [[2, 3, 5, 4]] [[1, 4, 2, 3, 5]]
[[1, 3, 5, 2, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 4, 3]]
[[1, 3, 5, 2, 4]] [[2, 3, 5, 4]] [[1, 5, 4, 3, 2]]
[[1, 4, 2, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 4, 2]]
[[1, 4, 2, 3, 5]] [[2, 3, 5, 4]] [[1, 2, 3, 5, 4]]
[[1, 3, 5, 4, 2]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 5, 4]]
[[1, 3, 5, 4, 2]] [[2, 3, 5, 4]] [[1, 5, 4, 2, 3]]
[[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 5, 4]]
[[1, 2, 3, 5, 4]] [[2, 3, 5, 4]] [[1, 3, 5, 4, 2]]
[[1, 3, 2, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 3, 4]]
[[1, 3, 2, 5, 4]] [[2, 3, 5, 4]] [[1, 5, 3, 4, 2]]
[[1, 2, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 4, 5]]
[[1, 2, 5, 3, 4]] [[2, 3, 5, 4]] [[1, 3, 4, 5, 2]]
[[1, 2, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 2, 3, 5]]
[[1, 2, 3, 4, 5]] [[2, 3, 5, 4]] [[1, 3, 5, 2, 4]]
[[1, 3, 4, 5, 2]] [[1, 2, 3, 5, 4]] [[1, 4, 3, 2, 5]]
[[1, 3, 4, 5, 2]] [[2, 3, 5, 4]] [[1, 5, 2, 4, 3]]
[[1,

[[1, 3, 5, 2]] [[2, 3, 5, 4]] [[1, 5, 4, 3]]
[[1, 2, 3, 5]] [[1, 2, 3, 5, 4]] [[2, 3, 5, 4]]
[[1, 2, 3, 5]] [[2, 3, 5, 4]] [[1, 3, 5, 4]]
[[2, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 4]]
[[2, 3, 5, 4]] [[2, 3, 5, 4]] [[2, 3, 5, 4]]
[[1, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 4]]
[[1, 3, 5, 4]] [[2, 3, 5, 4]] [[1, 5, 4, 2]]
[[1, 2, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 4]]
[[1, 2, 5, 4]] [[2, 3, 5, 4]] [[1, 3, 4, 2]]
[[1, 2, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 5]]
[[1, 2, 3, 4]] [[2, 3, 5, 4]] [[1, 3, 5, 2]]
[[1, 5, 2, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 4, 3]]
[[1, 5, 2, 4]] [[2, 3, 5, 4]] [[1, 4, 3, 2]]
[[2, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 3, 5]]
[[2, 3, 4, 5]] [[2, 3, 5, 4]] [[2, 4, 3, 5]]
[[1, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 4, 5]]
[[1, 5, 3, 4]] [[2, 3, 5, 4]] [[1, 4, 5, 2]]
[[3, 1], [5, 4]] [[1, 2, 3, 5, 4]] [[1, 4], [2, 5]]
[[3, 1], [5, 4]] [[2, 3, 5, 4]] [[1, 5], [2, 4]]
[[1, 4], [2, 5]] [[1, 2, 3, 5, 4]] [[1, 2], [3, 4]]
[[1, 4], [2, 5]] [[2, 3, 5, 4]] [[1, 2], [3, 4]]
[[1, 2], 

...

[[1, 5, 3, 2]] [[2, 3, 5, 4]] [[1, 4, 5, 3]]
[[2, 4, 5, 3]] [[1, 2, 3, 5, 4]] [[1, 4, 5, 3]]
[[2, 4, 5, 3]] [[2, 3, 5, 4]] [[2, 4, 5, 3]]
[[1, 4, 5, 3]] [[1, 2, 3, 5, 4]] [[1, 4, 5, 2]]
[[1, 4, 5, 3]] [[2, 3, 5, 4]] [[1, 2, 4, 5]]
[[1, 4, 5, 2]] [[1, 2, 3, 5, 4]] [[1, 4, 3, 2]]
[[1, 4, 5, 2]] [[2, 3, 5, 4]] [[1, 2, 4, 3]]
[[1, 4, 3, 2]] [[1, 2, 3, 5, 4]] [[1, 5, 3, 2]]
[[1, 4, 3, 2]] [[2, 3, 5, 4]] [[1, 2, 5, 3]]
[[1, 2, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 3]]
[[1, 2, 4, 3]] [[2, 3, 5, 4]] [[1, 3, 2, 5]]
[[1, 2, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 2, 3]]
[[1, 2, 4, 5]] [[2, 3, 5, 4]] [[1, 3, 2, 4]]
[[1, 4, 2, 3]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 2]]
[[1, 4, 2, 3]] [[2, 3, 5, 4]] [[1, 2, 3, 5]]
[[1, 3, 5, 2]] [[1, 2, 3, 5, 4]] [[2, 5, 4, 3]]
[[1, 3, 5, 2]] [[2, 3, 5, 4]] [[1, 5, 4, 3]]
[[2, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 3, 4]]
[[2, 5, 4, 3]] [[2, 3, 5, 4]] [[2, 5, 3, 4]]
[[1, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 5, 4, 3]] [[2, 3, 5, 4]] [[1, 4, 2, 5]]
[[1, 5, 2, 4]] [[1, 2, 3,

[[1, 3, 4, 5]] [[2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 4, 2, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 2]]
[[1, 4, 2, 5]] [[2, 3, 5, 4]] [[1, 2, 3, 4]]
[[1, 3, 4, 2]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 5]]
[[1, 3, 4, 2]] [[2, 3, 5, 4]] [[1, 5, 2, 3]]
[[1, 3, 2, 5]] [[1, 2, 3, 5, 4]] [[2, 5, 3, 4]]
[[1, 3, 2, 5]] [[2, 3, 5, 4]] [[1, 5, 3, 4]]
[[1, 5, 2, 3]] [[1, 2, 3, 5, 4]] [[2, 4, 3, 5]]
[[1, 5, 2, 3]] [[2, 3, 5, 4]] [[1, 4, 3, 5]]
[[2, 4, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 5, 4, 3]]
[[2, 4, 3, 5]] [[2, 3, 5, 4]] [[2, 5, 4, 3]]
[[1, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 5, 4, 3]] [[2, 3, 5, 4]] [[1, 4, 2, 5]]
[[2, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 3, 4]]
[[2, 5, 4, 3]] [[2, 3, 5, 4]] [[2, 5, 3, 4]]
[[1, 4, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 5, 4, 2]]
[[1, 4, 3, 5]] [[2, 3, 5, 4]] [[1, 2, 5, 4]]
[[1, 5, 4, 2]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 4]]
[[1, 5, 4, 2]] [[2, 3, 5, 4]] [[1, 4, 2, 3]]
[[1, 3, 2, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 3]]
[[1, 3, 2, 4]] [[2, 3, 5, 4]] [[1, 5, 3, 2]]
[[1, 2, 5, 3]] [[1, 2, 3,

[[1, 5], [2, 4]] [[2, 3, 5, 4]] [[1, 4], [2, 3]]
[[2, 1, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 5]]
[[2, 1, 3, 4]] [[2, 3, 5, 4]] [[1, 5, 2, 3]]
[[1, 3, 2, 5]] [[1, 2, 3, 5, 4]] [[2, 5, 3, 4]]
[[1, 3, 2, 5]] [[2, 3, 5, 4]] [[1, 5, 3, 4]]
[[2, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 5]]
[[2, 5, 3, 4]] [[2, 3, 5, 4]] [[2, 3, 4, 5]]
[[1, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 2, 5]]
[[1, 3, 4, 5]] [[2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 4, 2, 5]] [[1, 2, 3, 5, 4]] [[1, 3, 4, 2]]
[[1, 4, 2, 5]] [[2, 3, 5, 4]] [[1, 2, 3, 4]]
[[1, 3, 4, 2]] [[1, 2, 3, 5, 4]] [[1, 3, 2, 5]]
[[1, 3, 4, 2]] [[2, 3, 5, 4]] [[1, 5, 2, 3]]
[[1, 2, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 3, 5]]
[[1, 2, 3, 4]] [[2, 3, 5, 4]] [[1, 3, 5, 2]]
[[1, 2, 3, 5]] [[1, 2, 3, 5, 4]] [[2, 3, 5, 4]]
[[1, 2, 3, 5]] [[2, 3, 5, 4]] [[1, 3, 5, 4]]
[[2, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 3, 5, 4]]
[[2, 3, 5, 4]] [[2, 3, 5, 4]] [[2, 3, 5, 4]]
[[1, 3, 5, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 4]]
[[1, 3, 5, 4]] [[2, 3, 5, 4]] [[1, 5, 4, 2]]
[[1, 2, 5, 4]] [[1, 2

[[2, 5, 4, 3]] [[2, 3, 5, 4]] [[2, 5, 3, 4]]
[[1, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 4]]
[[1, 5, 4, 3]] [[2, 3, 5, 4]] [[1, 4, 2, 5]]
[[1, 5, 2, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 4, 3]]
[[1, 5, 2, 4]] [[2, 3, 5, 4]] [[1, 4, 3, 2]]
[[2, 3, 4, 5]] [[1, 2, 3, 5, 4]] [[1, 4, 3, 5]]
[[2, 3, 4, 5]] [[2, 3, 5, 4]] [[2, 4, 3, 5]]
[[1, 4, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 5, 4, 2]]
[[1, 4, 3, 5]] [[2, 3, 5, 4]] [[1, 2, 5, 4]]
[[2, 4, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 5, 4, 3]]
[[2, 4, 3, 5]] [[2, 3, 5, 4]] [[2, 5, 4, 3]]
[[1, 5, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 4, 5]]
[[1, 5, 3, 4]] [[2, 3, 5, 4]] [[1, 4, 5, 2]]
[[1, 5, 2, 3]] [[1, 2, 3, 5, 4]] [[2, 4, 3, 5]]
[[1, 5, 2, 3]] [[2, 3, 5, 4]] [[1, 4, 3, 5]]
[[2, 4, 1, 3, 5]] [[1, 2, 3, 5, 4]] [[1, 2, 5, 4, 3]]
[[2, 4, 1, 3, 5]] [[2, 3, 5, 4]] [[1, 5, 4, 3, 2]]
[[1, 2, 5, 4, 3]] [[1, 2, 3, 5, 4]] [[1, 5, 2, 3, 4]]
[[1, 2, 5, 4, 3]] [[2, 3, 5, 4]] [[1, 3, 4, 2, 5]]
[[1, 5, 2, 3, 4]] [[1, 2, 3, 5, 4]] [[1, 2, 4, 3, 5]]
[[1, 5, 2, 3, 4]] [[2, 3, 5, 4]] [[1, 4,

KeyboardInterrupt: 

In [ ]:
def nodeCcl(generators, node, cclList):
    for i in cclList:
        if node in i:
            return
    cclList.append(wildfire(generators,node))
    for j in generators:
        new = multiply(node, j)
        nodeCcl(generators, new, cclList)
        
def wildfire2(generators):
    while somethingThatHasntBeenDecidedYetHappens != "happening":
        
    cclList = conjClass(generators)
    for node in generators:
        nodeCcl(generators,node, cclList)
    return cclList